<a href="https://colab.research.google.com/github/bhargav23/NLP-Projects/blob/main/Faster_Sentiment_Analysis_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3 - Faster Sentiment Analysis

In the previous notebook we managed to achieve a decent test accuracy of ~84% using all of the common techniques used for sentiment analysis. In this notebook, we'll implement a model that gets comparable results whilst training significantly faster and using around half of the parameters. More specifically, we'll be implementing the "FastText" model from the paper [Bag of Tricks for Efficient Text Classification](https://arxiv.org/abs/1607.01759).

## Preparing Data

One of the key concepts in the FastText paper is that they calculate the n-grams of an input sentence and append them to the end of a sentence. Here, we'll use bi-grams. Briefly, a bi-gram is a pair of words/tokens that appear consecutively within a sentence. 

For example, in the sentence "how are you ?", the bi-grams are: "how are", "are you" and "you ?".

The `generate_bigrams` function takes a sentence that has already been tokenized, calculates the bi-grams and appends them to the end of the tokenized list.

In [1]:
!pip install -q kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"bhargav23","key":"75139d87df4b73aa37365c34ba610b07"}'}

In [3]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d snap/amazon-fine-food-reviews

 90% 217M/242M [00:01<00:00, 99.6MB/s]
100% 242M/242M [00:02<00:00, 120MB/s] 


In [5]:
!unzip amazon-fine-food-reviews.zip -d data

Archive:  amazon-fine-food-reviews.zip
  inflating: data/Reviews.csv        
  inflating: data/database.sqlite    
  inflating: data/hashes.txt         


In [6]:
DATA_PATH = 'data/Reviews.csv'

In [7]:
import pandas as pd
import numpy as np
import torch

In [8]:
odf = pd.read_csv('data/Reviews.csv')

In [9]:
df = odf[odf['Score'] != 3]
X = df['Text']
y_dict = {1:0, 2:0, 4:1, 5:1}
y = df['Score'].map(y_dict)

In [10]:
TEXT = pd.Series(X,name="Text")
train_df = pd.concat([TEXT, y], axis=1)

In [11]:
train_df.head()

,Text,Score
0,I have bought several of the Vitality canned d...,1
1,Product arrived labeled as Jumbo Salted Peanut...,0
2,This is a confection that has been around a fe...,1
3,If you are looking for the secret ingredient i...,0
4,Great taffy at a great price. There was a wid...,1


In [12]:
train_df.dtypes

Text     object
Score     int64
dtype: object

In [13]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(train_df, test_size=0.2, random_state=42, shuffle=True)

In [14]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 420651
Number of testing examples: 105163


In [15]:
import random
SEED = 1234
train_data, valid_data = train_test_split(train_data,random_state = random.seed(SEED))

In [16]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 315488
Number of validation examples: 105163
Number of testing examples: 105163


In [17]:
def generate_bigrams(x):
    n_grams = set(zip(*[x[i:] for i in range(2)]))
    for n_gram in n_grams:
        x.append(' '.join(n_gram))
    return x

As an example:

In [18]:
generate_bigrams(['This', 'film', 'is', 'terrible'])

['This', 'film', 'is', 'terrible', 'film is', 'This film', 'is terrible']

TorchText `Field`s have a `preprocessing` argument. A function passed here will be applied to a sentence after it has been tokenized (transformed from a string into a list of tokens), but before it has been numericalized (transformed from a list of tokens to a list of indexes). This is where we'll pass our `generate_bigrams` function.

As we aren't using an RNN we can't use packed padded sequences, thus we do not need to set `include_lengths = True`.

In [19]:
import torch
from torchtext import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', preprocessing = generate_bigrams)
SCORE = data.LabelField(dtype = torch.float)

In [20]:
fields = {'Text': ('t', TEXT), 'Score': ('s', SCORE)}

In [21]:
train_data.to_csv('train.csv')
valid_data.to_csv('valid.csv')
test_data.to_csv('test.csv')

In [22]:
train_data, valid_data, test_data = data.TabularDataset.splits(
                                        path = '',
                                        train = 'train.csv',
                                        validation = 'valid.csv',
                                        test = 'test.csv',
                                        format = 'csv',
                                        fields = fields,
                                        skip_header = False
)

In [23]:
print(vars(train_data[0]))

{'t': ['The', 'Trinidad', 'Hot', 'Habanero', 'Sauce', 'is', 'literally', 'the', 'best', 'of', 'hundreds', 'that', 'I', 'have', 'tried', '(', 'including', 'local', 'products', 'in', 'the', 'Caribbean', ';', 'just', 'wish', 'I', 'could', 'find', '"', 'Bob', "'s", '"', 'again', ')', '.', 'Actually', ',', 'the', 'best', 'was', '"', 'Vib', "'s", 'Caribbean', 'Heat', '"', 'that', 'was', 'sold', 'in', 'the', 'St.', 'Louis', 'area', 'years', 'ago', 'but', 'who', "'s", 'owner', 'and', 'recipe', 'has', 'apparently', 'been', 'lost', 'forever', '.', 'If', 'you', 'prefer', 'the', 'habanero', 'or', 'scotch', 'bonnet', 'pepper', ',', 'in', 'a', 'vinegar', 'base', ',', 'then', 'this', 'is', 'the', 'best', 'that', 'is', 'out', 'there', '.', 'A', 'little', 'shop', 'in', 'St.', 'Charles', ',', 'Missouri', '(', 'Figueros)has', 'hundreds', 'of', 'different', 'sauces', 'and', 'thousands', 'of', 'bottles', 'and', 'I', 'have', 'tried', 'many', 'over', 'the', 'years', '.', 'Trinidad', "'s", 'has', 'the', 'perf


As before, we load the IMDb dataset and create the splits.

In [24]:
#import random

#train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

#train_data, valid_data = train_data.split(random_state = random.seed(SEED))

Build the vocab and load the pre-trained word embeddings.

In [25]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

SCORE.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [07:17, 1.97MB/s]                           
100%|█████████▉| 398559/400000 [00:16<00:00, 24731.99it/s]

And create the iterators.

In [26]:
"""
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)
"""
BATCH_SIZE = 64
 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


 
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    sort_key = lambda x: len(x.t), #sort by s attribute (quote)
    batch_size=BATCH_SIZE,
    device=device)

## Build the Model

This model has far fewer parameters than the previous model as it only has 2 layers that have any parameters, the embedding layer and the linear layer. There is no RNN component in sight!

Instead, it first calculates the word embedding for each word using the `Embedding` layer (blue), then calculates the average of all of the word embeddings (pink) and feeds this through the `Linear` layer (silver), and that's it!

![](https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/assets/sentiment8.png?raw=1)

We implement the averaging with the `avg_pool2d` (average pool 2-dimensions) function. Initially, you may think using a 2-dimensional pooling seems strange, surely our sentences are 1-dimensional, not 2-dimensional? However, you can think of the word embeddings as a 2-dimensional grid, where the words are along one axis and the dimensions of the word embeddings are along the other. The image below is an example sentence after being converted into 5-dimensional word embeddings, with the words along the vertical axis and the embeddings along the horizontal axis. Each element in this [4x5] tensor is represented by a green block.

![](https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/assets/sentiment9.png?raw=1)

The `avg_pool2d` uses a filter of size `embedded.shape[1]` (i.e. the length of the sentence) by 1. This is shown in pink in the image below.

![](https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/assets/sentiment10.png?raw=1)

We calculate the average value of all elements covered by the filter, then the filter then slides to the right, calculating the average over the next column of embedding values for each word in the sentence. 

![](https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/assets/sentiment11.png?raw=1)

Each filter position gives us a single value, the average of all covered elements. After the filter has covered all embedding dimensions we get a [1x5] tensor. This tensor is then passed through the linear layer to produce our prediction.

In [27]:
import torch.nn as nn
import torch.nn.functional as F

class FastText(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_dim, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        
        self.fc = nn.Linear(embedding_dim, output_dim)
        
    def forward(self, text):
        
        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
                
        #embedded = [sent len, batch size, emb dim]
        
        embedded = embedded.permute(1, 0, 2)
        
        #embedded = [batch size, sent len, emb dim]
        
        pooled = F.avg_pool2d(embedded, (embedded.shape[1], 1)).squeeze(1) 
        
        #pooled = [batch size, embedding_dim]
                
        return self.fc(pooled)

As previously, we'll create an instance of our `FastText` class.

In [28]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
OUTPUT_DIM = 1
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = FastText(INPUT_DIM, EMBEDDING_DIM, OUTPUT_DIM, PAD_IDX)

Looking at the number of parameters in our model, we see we have about the same as the standard RNN from the first notebook and half the parameters of the previous model.

In [29]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,500,301 trainable parameters


And copy the pre-trained vectors to our embedding layer.

In [30]:
pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.3398,  0.2094,  0.4635,  ..., -0.2339,  0.4730, -0.0288],
        ...,
        [-1.7185,  1.8136,  1.3652,  ...,  0.5463, -0.4997,  0.5954],
        [-0.5423, -0.6678,  0.2622,  ..., -0.4284, -1.3810, -0.8909],
        [ 1.3802,  1.7709,  1.2272,  ..., -1.9781, -1.1305, -0.5816]])

Not forgetting to zero the initial weights of our unknown and padding tokens.

In [31]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

## Train the Model

Training the model is the exact same as last time.

We initialize our optimizer...

In [32]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

We define the criterion and place the model and criterion on the GPU (if available)...

In [33]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

100%|█████████▉| 398559/400000 [00:30<00:00, 24731.99it/s]

We implement the function to calculate accuracy...

In [34]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

We define a function for training our model...

**Note**: we are no longer using dropout so we do not need to use `model.train()`, but as mentioned in the 1st notebook, it is good practice to use it.

In [35]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.t).squeeze(1)
        
        loss = criterion(predictions, batch.s)
        
        acc = binary_accuracy(predictions, batch.s)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

We define a function for testing our model...

**Note**: again, we leave `model.eval()` even though we do not use dropout.

In [36]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.t).squeeze(1)
            
            loss = criterion(predictions, batch.s)
            
            acc = binary_accuracy(predictions, batch.s)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

As before, we'll implement a useful function to tell us how long an epoch takes.

In [37]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Finally, we train our model.

In [38]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 1m 23s
	Train Loss: 0.301 | Train Acc: 88.40%
	 Val. Loss: 0.380 |  Val. Acc: 93.25%
Epoch: 02 | Epoch Time: 1m 20s
	Train Loss: 0.166 | Train Acc: 93.70%
	 Val. Loss: 0.422 |  Val. Acc: 94.08%
Epoch: 03 | Epoch Time: 1m 21s
	Train Loss: 0.139 | Train Acc: 94.95%
	 Val. Loss: 0.436 |  Val. Acc: 94.60%
Epoch: 04 | Epoch Time: 1m 21s
	Train Loss: 0.123 | Train Acc: 95.60%
	 Val. Loss: 0.430 |  Val. Acc: 95.17%
Epoch: 05 | Epoch Time: 1m 21s
	Train Loss: 0.114 | Train Acc: 96.02%
	 Val. Loss: 0.445 |  Val. Acc: 95.30%


...and get the test accuracy!

The results are comparable to the results in the last notebook, but training takes considerably less time!

In [39]:
model.load_state_dict(torch.load('tut3-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.365 | Test Acc: 93.34%


## User Input

And as before, we can test on any input the user provides making sure to generate bigrams from our tokenized sentence.

In [40]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = generate_bigrams([tok.text for tok in nlp.tokenizer(sentence)])
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

An example negative review...

In [41]:
predict_sentiment(model, "This film is terrible")

1.0

An example positive review...

In [42]:
predict_sentiment(model, "This film is great")

0.0

## Next Steps

In the next notebook we'll use convolutional neural networks (CNNs) to perform sentiment analysis.